# Training classifier

This script can be used to train the classifier on original and adversarial samples.

In [21]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import time
import importlib
from copy import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading and transforming data into logits differences

The first step is transforming our dataframe into logits differences for each original and adversarial sentence. For this, it is required to execute the model for each sentence with substitutions as explained in the paper.

In [22]:
# Print available setups for testing
for i in os.listdir('../../Generating Adversarial Samples/Data'):
    if not i.startswith('.'): # Don't print system files
        print(i)

rotten-tomatoes_pwws_distilbert.csv
imdb_pwws_cnn.csv
ag-news_pwws_distilbert.csv
ag-news_pwws_cnn.csv
ag-news_pwws_lstm.csv
ag-news_kuleshov_distilbert.csv
ag-news_pwws_bert.csv
imdb_pwws_distilbert.csv
imdb_alzantot_distilbert.csv
rotten-tomatoes_alzantot_distilbert.csv
yelp-polarity_textfooler_bert.csv
imdb_kuleshov_distilbert.csv
ag-news_alzantot_distilbert.csv
imdb_pwws_bert.csv
imdb_pwws_lstm.csv
yelp-polarity_pwws_bert.csv
ag-news_textfooler_distilbert.csv
imdb_textfooler_distilbert.csv


In [10]:
# Select the configuration for training
test_config = 'imdb_pwws_distilbert.csv' # or 'agnews_pwws_distilbert.csv'

In [11]:
# Obtain model from test config
model_arch = test_config.replace(".csv", "").split('_')[-1]
dataset = test_config.split('_')[0]
print("Model architecture:", model_arch)
print("Dataset:", dataset)

Model architecture: distilbert
Dataset: imdb


In [12]:
def load_textattack_local_model(model_arch, dataset):
    
    def load_module_from_file(file_path):
        """Uses ``importlib`` to dynamically open a file and load an object from
        it."""
        temp_module_name = f"temp_{time.time()}"

        spec = importlib.util.spec_from_file_location(temp_module_name, file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        return module
    
    m = load_module_from_file(f'../{model_arch}_{dataset}_textattack.py')
    model = getattr(m, 'model')
    
    return model, None

In [13]:
def load_hugging_face_model(model_arch, dataset):
    # Import the model used for generating the adversarial samples.
    # Correctly, set up imports, model and tokenizer depending on the model you generated the samples on.
    
    if model_arch == 'distilbert':
        from transformers import DistilBertConfig as config, DistilBertTokenizer as tokenizer, AutoModelForSequenceClassification as auto_model
    elif model_arch == 'bert':
        from transformers import BertConfig as config, BertTokenizer as tokenizer, AutoModelForSequenceClassification as auto_model
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = tokenizer.from_pretrained(f"textattack/{model_arch}-base-uncased-{dataset}")
    model = auto_model.from_pretrained(f"textattack/{model_arch}-base-uncased-{dataset}").to(device)
    
    return model, tokenizer

In [14]:
# Models available in hugging-face are executed differently from LSTM and CNN. Choose automatically the configuration and load model + tokenizer.
textattack_local_models = ['lstm', 'cnn']

if model_arch in textattack_local_models:
    hugging_face_model = False
    model, tokenizer = load_textattack_local_model(model_arch, dataset)

else:
    hugging_face_model = True
    model, tokenizer = load_hugging_face_model(model_arch, dataset)

# Loading data

Read into a dataframe your original and adversarial samples.

In [15]:
# Read the desired csv file previously generated
df = pd.read_csv(f'../../Generating Adversarial Samples/Data/{test_config}', index_col=0)
df.shape

(28492, 12)

In [16]:
# Select first entries. Only 3000 will be used but we leave room for false adversarial sentences that will be filtered out later and test set. We reduce size because computations are expensive.
# In real setup, the whole file was considered and fixed train and test sets were produced.
df = df.head(7000)

In [17]:
# Create batches of non-adversarial sentences
# For big models such as BERT, we must divide our input in smaller batches.
n = 256 # Size of each batch.
batches = [list(df.original_text.values)[i:i + n] for i in range(0, len(df.original_text.values), n)]

In [20]:
batches[0][0]

'I recently purchased the complete American Gothic series on DVD and it lived up to my memories of it. I was very grateful to be able to view for the first time episodes that were never televised. I loved "Ring of Fire" in particular of the stories I hadn\'t seen the first time around.  Gary Cole is fantastic as "evil, sexy" Lucas Buck. Lucas Black as Caleb is also a superb player. I thought Brenda Bakke as Selena Coombs was also superb in her portrayal. In fact, the whole cast was fantastically talented and had great chemistry with each other.  It\'s a shame the series was screwed by the network (in collusion with a burgeoning group of censors) because it was truly designed for adult viewing. A mixture of comedy, tragedy, farce, satire, Gothic romance and horror genres, it offered brilliant characterizations supported by acting at the genius level.  I had the most tremendous lust for the devil for once in my life. Long live Gary Cole (Sheriff Lucas Buck), the most luscious "fallen ang

In [ ]:
# Generate predictions for all non-adversarial sentences in our dataset
outputs = []

if hugging_face_model is True: # Use tokenizer and hugging face pipeline
    for b in batches: 
        input = tokenizer(b, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            output = model(**input)
            outputs.append(output.logits.cpu().numpy())
            del input
            torch.cuda.empty_cache()

else: # Use local model by simply predicting without tokenization
    for b in batches: 
        output = model(b)
        outputs.append(output)

In [ ]:
# Obtain non-adversarial predictions
outputs_flatten = [item for sublist in outputs for item in sublist]
predictions = [np.argmax(i) for i in outputs_flatten]

# Include prediction for these classes in our DataFrame
df['original_class_predicted'] = predictions

In [ ]:
# Repeat process for adversarial sentences
n = 256
batches = [list(df.adversarial_text.values)[i:i + n] for i in range(0, len(df.adversarial_text.values), n)]

In [ ]:
# Generate predictions for all non-adversarial sentences in our dataset
outputs = []

if hugging_face_model is True: # Use tokenizer and hugging face pipeline
    for b in batches: 
        input = tokenizer(b, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            output = model(**input)
            outputs.append(output.logits.cpu().numpy())
            del input
            torch.cuda.empty_cache()

else: # Use local model by simply predicting without tokenization
    for b in batches: 
        output = model(b)
        outputs.append(output)

In [ ]:
# Obtain adversarial predictions
outputs_flatten = [item for sublist in outputs for item in sublist]
predictions = [np.argmax(i) for i in outputs_flatten]

# Include prediction for these classes in our DataFrame
df['adversarial_class_predicted'] = predictions

In [ ]:
# Select only those sentences for which there was actually a change in the prediction
correct = df[(df['original_class_predicted'] != df['adversarial_class_predicted'])]

In [ ]:
# Update dataframe and keep only adversarial samples
df = correct

# Obtain logits
Once we have the predictions and actually adversarial sentences, we generate the logits differences

In [ ]:
original_samples = df.original_text.values
adversarial_samples = df.adversarial_text.values

In [ ]:
# Concatenate all original samples and their predictions
x = np.concatenate((original_samples, adversarial_samples))
y = np.concatenate((np.zeros(len(original_samples)), np.ones(len(adversarial_samples))))

In [ ]:
def obtain_logits(samples, batch_size, model, tokenizer):
    """
    For given samples and model, compute prediction logits.
    Input data is splitted in batches.
    """
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]
    logits = []

    for i, b in enumerate(batches):
        print("{}/{}".format(i+1, len(batches)))
        if hugging_face_model:
            with torch.no_grad():
                input = tokenizer(list(b), return_tensors="pt", padding=True, truncation=True).to(device)
                logits.append(model(**input).logits.cpu().numpy())
        else:
            logits.append(model(b))

    return logits

In [ ]:
# Compute logits for original sentences
batch_size = 350
original_logits = obtain_logits(original_samples, batch_size, model, tokenizer)
original_logits = np.concatenate(original_logits).reshape(-1, original_logits[0].shape[1])

1/1


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Compute logits for adversarial sentences
batch_size = 350
adversarial_logits = obtain_logits(adversarial_samples, batch_size, model, tokenizer)
adversarial_logits = np.concatenate(adversarial_logits).reshape(-1, adversarial_logits[0].shape[1])

1/1


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Concatenate all logits
logits = np.concatenate((original_logits, adversarial_logits))

In [ ]:
# Shuffle data
import random
c = list(zip(x, y, logits))
random.shuffle(c)
x, y, logits = zip(*c)

## Computing logits difference

This is a key step implemented. The main idea is:
* For each sentence, replace each word by the `[UNK]` token and compute prediction logits
* Using these logits, we can easily compute the saliency of the word as presented in the report.
* Then, we sort words by descending saliency.
* Finally, compute logits difference for each replacement. This difference is computed as `Logit from class predicted for the whole sentence - Highest remaining logit`

More details on these derivations are found in the paper.

In [ ]:
def compute_logits_difference(x, logits, y, model, tokenizer, idx, max_sentence_size=512):
    n_classes = len(logits[idx])
    predicted_class = np.argmax(logits[idx]) # Predicted class for whole sentence using previously computed logits
    class_logit = logits[idx][predicted_class] # Store this origianl prediction logit

    split_sentence = x[idx].split(' ')[:max_sentence_size] # The tokenizer will only consider 512 words so we avoid computing innecessary logits

    new_sentences = []

    # Here, we replace each word by [UNK] and generate all sentences to consider
    for i, word in enumerate(split_sentence):
        new_sentence = copy(split_sentence)
        new_sentence[i] = '[UNK]'
        new_sentence = ' '.join(new_sentence)
        new_sentences.append(new_sentence)

    # We cannot run more than 350 predictions simultaneously because of resources.
    # Split in batches if necessary.
    # Compute logits for all replacements.
    if len(new_sentences) > 200:
        logits = []
        batches = [new_sentences[i:i + 200] for i in range(0, len(new_sentences), 200)]
        for b in batches:
            if hugging_face_model: # Use hugging face predictions
                batch = tokenizer(b, return_tensors="pt", padding=True, truncation=True).to(device)
                with torch.no_grad():
                    logits.append(model(**batch).logits)
            else:
                logits.append(model(b).to(device))
      
        if hugging_face_model:
            logits = torch.cat(logits)
        else:
            logits = np.concatenate( logits, axis=0 )
            logits = torch.Tensor(logits)
    
    else: # There's no need to split in batches
        if hugging_face_model:
            batch = tokenizer(new_sentences, return_tensors="pt", padding=True, truncation=True).to(device)
            with torch.no_grad():
                logits = model(**batch).logits
            del batch
        else:
            logits = model(new_sentences)
            logits = torch.Tensor(logits)


    # Compute saliency
    saliency = (class_logit - logits[:,predicted_class]).reshape(-1, 1)

    # Append to logits for sorting
    data = torch.cat((logits, saliency), 1)

    # Sort by descending saliency
    data = torch.stack(sorted(data, key=lambda a: a[n_classes], reverse=True))

    # Remove saliency
    data = data[:, :n_classes]

    # Fix order: originallly predicted class, other classes
    order = [predicted_class] + [i for i in range(n_classes) if i!=predicted_class]
    data = torch.index_select(data, 1, torch.LongTensor(order).to(device))

    # Compute difference between predicted class (always first column) and higher remaining logit
    data = data[:, :1].flatten() - torch.max(data[:, 1:], dim=1).values.flatten()

    del saliency
    torch.cuda.empty_cache()

    # Return only logits difference
    return data.reshape(-1, 1), torch.Tensor([y[idx]]).to(device)

In [ ]:
def compute_logits_difference_padding(x, logits, y, model, tokenizer, idx, target_size=512):
    """
    This function provides a wrapper for compute_logits_difference and includes padding to computations.
    """
    data, y = compute_logits_difference(x, logits, y, model, tokenizer, idx, target_size)
    data_size = min(512, data.shape[0])
    target = torch.zeros(target_size, 1).to(device)
    target[:data_size, :] = data

    return target, y

In [ ]:
from torch.utils.data import Dataset, DataLoader
import sys
from torch.autograd import Variable

class Text(Dataset):
    """
    Dataloader following torch details. Each time we get an item, we will compute
    the logits difference.
    """
    def __init__(self, x , logits, y, model, tokenizer, train=True, max_sentence_size=512):
        self.logits = logits
        self.y = y
        self.x = x
        self.model = model
        self.tokenizer = tokenizer
        self.max_sentence_size = max_sentence_size

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data, y = compute_logits_difference_padding(self.x, self.logits, self.y, self.model, self.tokenizer, idx, self.max_sentence_size)
        data = data[:, :1].unsqueeze(0)

        return data, y, self.x[idx]

In [ ]:
# Create the dataloader
train_ds = Text(x, logits, y, model, tokenizer)
train_loader = DataLoader(dataset=train_ds, batch_size=256, shuffle=True)

In [ ]:
# Define the target DataFrame to structure our data.
# It has a column for each input dimension (up to 512) and 
# it also includes whether it is adversarial or not (y_label) and the sentence from which the logits where extracted

data_train = pd.DataFrame(columns=[i for i in range(512)]+['y_label', 'sentence'])

In [ ]:
# Generate logits difference by running the loader.
for i, (data, y_label, sentence) in enumerate(train_loader):
    print("{}/{} - {}\n".format(i, len(train_loader), i/len(train_loader)))
    for v in range(len(data)):
        # Structure data and include in dataframe
        row = np.append(data[v].cpu().numpy().reshape(1,-1), np.array([y_label[v].item(), sentence[v]]))
        data_train = data_test.append(pd.DataFrame([row], columns=list(data_test)), ignore_index=True)

0/1 - 0.0



In [ ]:
# Divide train and test set
data_train = data_train.head(3000)
data_test = data_train.tail(len(data_train)-3001)

# Model training and comparison

We train different models and compare their performance.

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model using best parameters found
model = RandomForestClassifier(n_estimators=1600,
                               min_samples_split=10,
                               min_samples_leaf=2,
                               max_features='auto',
                               max_depth=None, 
                               bootstrap = True)
# Fit on training data
model.fit(x, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Actual class predictions
rf_predictions = model.predict(x_test)

In [ ]:
np.sum(rf_predictions==y_test)/len(y_test)

0.914

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, rf_predictions, digits=3))
print(confusion_matrix(y_test, rf_predictions))

              precision    recall  f1-score   support

         0.0      0.932     0.891     0.911       248
         1.0      0.897     0.937     0.917       252

    accuracy                          0.914       500
   macro avg      0.915     0.914     0.914       500
weighted avg      0.915     0.914     0.914       500

[[221  27]
 [ 16 236]]


### XGBoost

Best performing model. Hyperparamter tuning done with Dataiku.

In [ ]:
import xgboost as xgb

In [ ]:
xgb_classifier = xgb.XGBClassifier(
                    max_depth=3,
                    learning_rate=0.34281802,
                    gamma=0.6770816,
                    min_child_weight=2.5520658,
                    max_delta_step=0.71469694,
                    subsample=0.61460966,
                    colsample_bytree=0.73929816,
                    colsample_bylevel=0.87191725,
                    reg_alpha=0.9064181,
                    reg_lambda=0.5686102,
                    n_estimators=29,
                    silent=0,
                    nthread=4,
                    scale_pos_weight=1.0,
                    base_score=0.5,
                    missing=None,
                  )

In [ ]:
xgb_classifier.fit(x, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.87191725,
              colsample_bynode=1, colsample_bytree=0.73929816, gamma=0.6770816,
              learning_rate=0.34281802, max_delta_step=0.71469694, max_depth=3,
              min_child_weight=2.5520658, missing=None, n_estimators=29,
              n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0.9064181, reg_lambda=0.5686102, scale_pos_weight=1.0,
              seed=1337, silent=0, subsample=0.61460966, verbosity=1)

In [ ]:
xgb_predictions = xgb_classifier.predict(x_test)

In [ ]:
print(classification_report(y_test, xgb_predictions, digits=3))
print(confusion_matrix(y_test, xgb_predictions))

              precision    recall  f1-score   support

         0.0      0.949     0.895     0.921       248
         1.0      0.902     0.952     0.927       252

    accuracy                          0.924       500
   macro avg      0.925     0.924     0.924       500
weighted avg      0.925     0.924     0.924       500

[[222  26]
 [ 12 240]]


### AdaBoost classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier()

In [ ]:
abc.fit(x, y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [ ]:
abc_predictions = abc.predict(x_test)

In [ ]:
np.sum(abc_predictions==y_test)/len(y_test)

0.918

In [ ]:
print(classification_report(y_test, abc_predictions, digits=3))
print(confusion_matrix(y_test, abc_predictions))

              precision    recall  f1-score   support

         0.0      0.956     0.875     0.914       248
         1.0      0.886     0.960     0.922       252

    accuracy                          0.918       500
   macro avg      0.921     0.918     0.918       500
weighted avg      0.921     0.918     0.918       500

[[217  31]
 [ 10 242]]


### LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
parameters = {
    'objective': 'binary',
    'application': 'binary',
    'metric': ['binary_logloss'],
    'num_leaves': 35,
    'learning_rate': 0.13,
    'verbose': 1
}

In [ ]:
train_data = lgb.Dataset(x, label=y)
test_data = lgb.Dataset(x_test, label=y_test)

In [ ]:
lgbm_classifier = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=300)

[1]	valid_0's binary_logloss: 0.597939
[2]	valid_0's binary_logloss: 0.525612
[3]	valid_0's binary_logloss: 0.469367
[4]	valid_0's binary_logloss: 0.423547
[5]	valid_0's binary_logloss: 0.387778
[6]	valid_0's binary_logloss: 0.357721
[7]	valid_0's binary_logloss: 0.331829
[8]	valid_0's binary_logloss: 0.310683
[9]	valid_0's binary_logloss: 0.293977
[10]	valid_0's binary_logloss: 0.280445
[11]	valid_0's binary_logloss: 0.268066
[12]	valid_0's binary_logloss: 0.257237
[13]	valid_0's binary_logloss: 0.248693
[14]	valid_0's binary_logloss: 0.241157
[15]	valid_0's binary_logloss: 0.23551
[16]	valid_0's binary_logloss: 0.230435
[17]	valid_0's binary_logloss: 0.22547
[18]	valid_0's binary_logloss: 0.223468
[19]	valid_0's binary_logloss: 0.22089
[20]	valid_0's binary_logloss: 0.219927
[21]	valid_0's binary_logloss: 0.218465
[22]	valid_0's binary_logloss: 0.21647
[23]	valid_0's binary_logloss: 0.214435
[24]	valid_0's binary_logloss: 0.213436
[25]	valid_0's binary_logloss: 0.21353
[26]	valid_0's

In [ ]:
y_hat = lgbm_classifier.predict(x_test)

In [ ]:
y_hat.round()

array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0.

In [ ]:
np.sum(y_hat.round()==y_test)/len(y_test)

0.92

In [ ]:
print(classification_report(y_test, y_hat.round(), digits=3))
print(confusion_matrix(y_test, y_hat.round()))

              precision    recall  f1-score   support

         0.0      0.933     0.903     0.918       248
         1.0      0.908     0.937     0.922       252

    accuracy                          0.920       500
   macro avg      0.921     0.920     0.920       500
weighted avg      0.920     0.920     0.920       500

[[224  24]
 [ 16 236]]


### SVM

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(C=9.0622635,
          kernel='rbf',
          gamma='scale',
          coef0=0.0,
          tol=0.001,
          probability=True,
          max_iter=-1)

In [ ]:
svm_clf.fit(x, y)

SVC(C=9.0622635, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
svm_pred = svm_clf.predict(x_test)

In [ ]:
np.sum(svm_pred.round()==y_test)/len(y_test)

0.92

In [ ]:
print(classification_report(y_test, svm_pred.round(), digits=3))
print(confusion_matrix(y_test, svm_pred.round()))

              precision    recall  f1-score   support

         0.0      0.944     0.891     0.917       248
         1.0      0.898     0.948     0.923       252

    accuracy                          0.920       500
   macro avg      0.921     0.920     0.920       500
weighted avg      0.921     0.920     0.920       500

[[221  27]
 [ 13 239]]


### Perceptron NN

In [ ]:
from torch.utils.data import Dataset, DataLoader
import sys
from torch.autograd import Variable

class Text(Dataset):
    def __init__(self, x , y):
        self.y = y
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data = torch.tensor(self.x[idx].astype('float32')).to(device)
        y = torch.tensor(self.y[idx].astype('float32')).unsqueeze(0).to(device)
        return data, y

In [ ]:
train_ds = Text(x, y)
train_loader = DataLoader(dataset=train_ds, batch_size=128, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BasicModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BasicModel, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim  = output_dim

        self.fc1 = torch.nn.Linear(self.input_dim, self.hidden_dim)
        self.fc2 = torch.nn.Linear(self.hidden_dim, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return self.sigmoid(x)

In [ ]:
basic_classifier = BasicModel(input_dim=512*1, hidden_dim=50, output_dim=1).to(device)
c = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(basic_classifier.parameters(), lr=0.001)

train_loss_history = []
val_acc_history = []

In [ ]:
iter_per_epoch = len(train_loader)
num_epochs = 3
initial_epoch = 1
log_nth = 2
storing_frequency = 15
checkpoints_path = "/content/drive/MyDrive/ExplainableAI/Model/Saliency/checkpoints"

for epoch in range(initial_epoch, initial_epoch+num_epochs):
    basic_classifier.train()
    epoch_losses = []
    for i, (data, y_label) in enumerate(train_loader):
      optimizer.zero_grad()
      out = basic_classifier(data)
      loss = c(out, y_label)
      epoch_losses.append(loss.item())
      loss.backward()
      optimizer.step()

      if (i+1) % log_nth == 0:        
          print ('Epoch [{}/{}], Step [{}/{}], Loss for last {} batches: {:.4f}' 
                  .format(epoch, num_epochs, i+1, iter_per_epoch, log_nth, np.mean(np.array(epoch_losses[-log_nth:]))))
          #print_time()
      
      if (i+1) % storing_frequency == 0:        
          print('Storing with loss for last {} batches = {}'.format(storing_frequency, np.mean(np.array(epoch_losses[-storing_frequency:]))))
          #print_time()
          #torch.save(basic_classifier.state_dict(), checkpoints_path+"/final_model_epoch_{}_{}.checkpoint".format(epoch, i+1))
  
    # Store after whole epoch
    print ('Epoch [{}/{}] finished with loss = {:.4f}'.format(epoch, num_epochs, np.mean(np.array(epoch_losses))))
    #torch.save(basic_classifier.state_dict(), checkpoints_path+"/final_model_epoch_{}.checkpoint".format(epoch))

Epoch [1/3], Step [2/32], Loss for last 2 batches: 0.6663
Epoch [1/3], Step [4/32], Loss for last 2 batches: 0.6241
Epoch [1/3], Step [6/32], Loss for last 2 batches: 0.6275
Epoch [1/3], Step [8/32], Loss for last 2 batches: 0.6386
Epoch [1/3], Step [10/32], Loss for last 2 batches: 0.6317
Epoch [1/3], Step [12/32], Loss for last 2 batches: 0.6243
Epoch [1/3], Step [14/32], Loss for last 2 batches: 0.6200
Storing with loss for last 15 batches = 0.6331382433573405
Epoch [1/3], Step [16/32], Loss for last 2 batches: 0.6284
Epoch [1/3], Step [18/32], Loss for last 2 batches: 0.6207
Epoch [1/3], Step [20/32], Loss for last 2 batches: 0.6145
Epoch [1/3], Step [22/32], Loss for last 2 batches: 0.6066
Epoch [1/3], Step [24/32], Loss for last 2 batches: 0.6115
Epoch [1/3], Step [26/32], Loss for last 2 batches: 0.6225
Epoch [1/3], Step [28/32], Loss for last 2 batches: 0.5810
Epoch [1/3], Step [30/32], Loss for last 2 batches: 0.5960
Storing with loss for last 15 batches = 0.6086814324061076
E

In [ ]:
nn_pred = basic_classifier(torch.tensor(x_test.astype('float32')).to(device))

In [ ]:
nn_pred = nn_pred.flatten().detach().cpu().numpy().round()

In [ ]:
np.sum(nn_pred==y_test)/len(y_test)

0.904

In [ ]:
print(classification_report(y_test, nn_pred, digits=3))
print(confusion_matrix(y_test, nn_pred))

              precision    recall  f1-score   support

         0.0      0.885     0.927     0.906       248
         1.0      0.925     0.881     0.902       252

    accuracy                          0.904       500
   macro avg      0.905     0.904     0.904       500
weighted avg      0.905     0.904     0.904       500

[[230  18]
 [ 30 222]]
